---
title: Knuckledragger Solvers for Terence Tao's Equational Reasoning Challenge
date: 2024-09-26
---

Terence Tao has put forward an interesting equational reasoning challenge.

- https://mathstodon.xyz/@tao/110736805384878353
- https://terrytao.wordpress.com/2024/09/25/a-pilot-project-in-universal-algebra-to-explore-new-ways-to-collaborate-and-use-machine-assistance/
- https://mathoverflow.net/questions/450890/is-there-an-identity-between-the-commutative-identity-and-the-constant-identity/450905#450905
- https://github.com/teorth/equational_theories

It is reminiscent of a famous success story of automated theorem proving, proving the robbins algebras are boolean https://www.cs.unm.edu/~mccune/papers/robbins/ a previously unsolved conjecture which was shown by Bill McCune's EQP solver in 1996. Hopefully solvers are a lot more powerful now.

I've been fiddling with building a semi-automated proof assistant in python called Knuckledragger.  https://github.com/philzook58/knuckledragger 

Knuckledragger is structured around Z3, but recently I've been bolting in the various automated solvers. Z3 is excellent at many things (first class at the grounded problems one often encoutners in software verification), but weaker at quantifier reasoning or equational reasoning.

For these, Vampire, eprover, Twee, Zipperposition, and Prover9 are good candidates. I judge this somewhat by examining the CASC https://tptp.org/CASC/J12/ results

What I've done is built a pretty printer to the first order and higher order TPTP format. This printer is wrapper around by a Solver object that mimics exactly Z3's python api so that I can swap it in.

Even if you don't buy into what I'm generally doing in Knuckledragger (a system to weave these results kind of Hilbert style into larger proofs), I think having easy installation and programmatic access to the solvers in python could be quite useful.

Access to the solvers is not new. Isabelle https://isabelle.in.tum.de/ and Why3 https://www.why3.org/   also both have these solvers accessible if you prefer.

The TPTP organization https://www.tptp.org/ , largely orchestrated by Geoff Sutcliffe, has many resources, but especially System on TPTP https://tptp.org/cgi-bin/SystemOnTPTP , which lets you submit problem files online without installing these solvers.

One big advantage of python is the massive preexisting ecosystem and infrastructure. Case in point. You can try this blog post out on Google collab quite easily by clicking this link https://colab.research.google.com/github/philzook58/philzook58.github.io/blob/master/pynb/2024-09-26-tao_algebra.ipynb

Running this cell will install Knuckledragger and the external solvers. It'll take a good 5 minutes to do all that compiling. Have faith. It could ber better when I ship a binary for eprover.

In [ ]:
%%bash
git clone https://github.com/philzook58/knuckledragger.git
cd knuckledragger
python3 -m pip install -e .

In [ ]:
! cd knuckledragger && ./install.sh # get and build external solvers

I had to restart the jupyter kernel on colab for the `kdrag` package to be found. 

In [ ]:
import os
os._exit(00)

Alright! Now we're ready to go!

Currently I have:
- Vampire using SMTlib input https://vprover.github.io/
- Higher Order Vampire
- Eprover-HO https://github.com/eprover/eprover
- Twee https://nick8325.github.io/twee/
- Zipperposition - https://github.com/sneeuwballen/zipperposition
- nanoCoP-i - an intuitionstic prover. I'm excited for this one
- Leo-III

I've always been rather impressed by Twee's proof output ,so let us show that.

In [4]:
import kdrag as kd
import kdrag.solvers as solvers
import kdrag.smt as smt # Literally a reexporting of z3

T = smt.DeclareSort("T")
x,y,z = smt.Consts("x y z", T)
mul = smt.Function("mul", T, T, T)
kd.notation.mul.define([x,y], mul(x,y)) # This gives us multiplication notation.

s = solvers.TweeSolver()
s.add(smt.ForAll([x,y,z], (x * x) * y == y * x))
s.add(smt.Not(smt.ForAll([x,y], x * y == y * x))) # Negate the theorem to be proved for a refutation
s.set("timeout", 1000)
print(s.check())
print(s.proof().decode())

unsat
Here is the input problem:
  Axiom 1 (flattening): mul_8e3 = mul_8e(y_91, x_92).
  Axiom 2 (flattening): mul_8e2 = mul_8e(x_92, y_91).
  Axiom 3 (ax_0): mul_8e(mul_8e(X, X), Y) = mul_8e(Y, X).
  Goal 1 (ax_1): mul_8e(x_92, y_91) = mul_8e(y_91, x_92).

1. mul_8e(y_91, x_92) -> mul_8e3
2. mul_8e(x_92, y_91) -> mul_8e2
3. mul_8e(mul_8e(X, X), Y) -> mul_8e(Y, X)
4. mul_8e(X, mul_8e(Y, Y)) -> mul_8e(X, Y)
5. mul_8e(X, Y) <-> mul_8e(Y, X)
6. mul_8e2 -> mul_8e3

% SZS status Unsatisfiable

% SZS output start Proof
Axiom 1 (ax_0): mul_8e(mul_8e(X, X), Y) = mul_8e(Y, X).

Goal 1 (ax_1): mul_8e(x_92, y_91) = mul_8e(y_91, x_92).
Proof:
  mul_8e(x_92, y_91)
= { by axiom 1 (ax_0) R->L }
  mul_8e(mul_8e(y_91, y_91), x_92)
= { by axiom 1 (ax_0) R->L }
  mul_8e(mul_8e(x_92, x_92), mul_8e(y_91, y_91))
= { by axiom 1 (ax_0) R->L }
  mul_8e(mul_8e(mul_8e(y_91, y_91), mul_8e(y_91, y_91)), mul_8e(x_92, x_92))
= { by axiom 1 (ax_0) }
  mul_8e(mul_8e(mul_8e(y_91, y_91), y_91), mul_8e(x_92, x_92))
= { b

You can see the file actually being generated. I have not yet turned on mangling the filename. You can see the constants do need to be mangled with a unique identifier

In [19]:
! cat /tmp/twee.p

% TPTP file generated by Knuckledragger
tff(t_type, type, t : $tType ).
% Declarations
tff(mul_decl, type, mul_8e : t * t > t).
% Axioms and assertions
tff(ax_0, axiom, (![Z_90:t, Y_91:t, X_92:t] : (mul_8e(mul_8e(X_92, X_92), Y_91) = mul_8e(Y_91, X_92)))).
tff(ax_1, axiom, ~((![Y_91:t, X_92:t] : (mul_8e(X_92, Y_91) = mul_8e(Y_91, X_92))))).


I also added the ability to access the solver directly. This has been useful for debugging and testing, but could be useful for other purposes

In [21]:
! python3 -m kdrag.solvers twee /tmp/twee.p

Here is the input problem:
  Axiom 1 (flattening): mul_8e3 = mul_8e(y_91, x_92).
  Axiom 2 (flattening): mul_8e2 = mul_8e(x_92, y_91).
  Axiom 3 (ax_0): mul_8e(mul_8e(X, X), Y) = mul_8e(Y, X).
  Goal 1 (ax_1): mul_8e(x_92, y_91) = mul_8e(y_91, x_92).

1. mul_8e(y_91, x_92) -> mul_8e3
2. mul_8e(x_92, y_91) -> mul_8e2
3. mul_8e(mul_8e(X, X), Y) -> mul_8e(Y, X)
4. mul_8e(X, mul_8e(Y, Y)) -> mul_8e(X, Y)
5. mul_8e(X, Y) <-> mul_8e(Y, X)
6. mul_8e2 -> mul_8e3

The conjecture is true! Here is a proof.

Axiom 1 (ax_0): mul_8e(mul_8e(X, X), Y) = mul_8e(Y, X).

Goal 1 (ax_1): mul_8e(x_92, y_91) = mul_8e(y_91, x_92).
Proof:
  mul_8e(x_92, y_91)
= { by axiom 1 (ax_0) R->L }
  mul_8e(mul_8e(y_91, y_91), x_92)
= { by axiom 1 (ax_0) R->L }
  mul_8e(mul_8e(x_92, x_92), mul_8e(y_91, y_91))
= { by axiom 1 (ax_0) R->L }
  mul_8e(mul_8e(mul_8e(y_91, y_91), mul_8e(y_91, y_91)), mul_8e(x_92, x_92))
= { by axiom 1 (ax_0) }
  mul_8e(mul_8e(mul_8e(y_91, y_91), y_91), mul_8e(x_92, x_92))
= { by axiom 1 (ax_0) 

# Vampire

Vampire very often wins many categories of the CASC competition. It is one of the best automated solvers of it's kind.

In [14]:
# This one actually uses the smt printer
s = solvers.VampireSolver()
s.add(smt.ForAll([x,y,z], (x * x) * y == y * x))
s.add(smt.Not(smt.ForAll([x,y], x * y == y * x)))
s.set("timeout", 1000)
print(s.check())

unsat


In [16]:
!cat /tmp/vampire.smt2

(set-logic ALL)
(declare-sort T 0)
;;declarations
(declare-fun mul (T T) T)
;;axioms
(assert (forall ((x T) (y T) (z T)) (= (mul (mul x x) y) (mul y x))))
(assert (not (forall ((x T) (y T)) (= (mul x y) (mul y x)))))
(check-sat)


# Higher Order

The next two solvers also support higher order reasoning (they accept lambda terms in their input). The recent progress in that direction is very exciting. I've  got blog posts to write on how to use these sorts of things.

In [12]:
s = solvers.VampireTHFSolver()
s.add(smt.ForAll([x,y,z], (x * x) * y == y * x))
s.add(smt.Not(smt.ForAll([x,y], x * y == y * x)))
s.set("timeout", 1000)
print(s.check())
print(s.proof().decode())

unsat
% WARNING: time unlimited strategy and instruction limiting not in place - attemping to translate instructions to time
% lrs+1002_1:1_au=on:av=off:fd=off:fde=unused:ntd=on:sos=on:sp=frequency:ss=axioms:i=782:si=on:rtra=on_0 on vampire for (3ds/782Mi)
% Refutation not found, incomplete strategy
% ------------------------------
% Version: Vampire 4.8 HO - Sledgehammer schedules (2023-10-19)
% Termination reason: Refutation not found, incomplete strategy


% Memory used [KB]: 895
% Time elapsed: 0.001 s
% ------------------------------
% ------------------------------
% WARNING: time unlimited strategy and instruction limiting not in place - attemping to translate instructions to time
% lrs+1002_2:3_avsq=on:fde=none:nwc=3.0:prag=on:sac=on:sd=2:sgt=32:sos=on:sp=unary_frequency:ss=axioms:i=754:si=on:rtra=on_0 on vampire for (3ds/754Mi)
% Refutation not found, incomplete strategy
% ------------------------------
% Version: Vampire 4.8 HO - Sledgehammer schedules (2023-10-19)
% Terminat

In [17]:
! cat /tmp/vampire.p

% TPTP file generated by Knuckledragger
thf(t_type, type, t : $tType ).
% Declarations
thf(mul_decl, type, mul_8e : t > t > t).
% Axioms and assertions
thf(ax_0, axiom, (![Z_90:t, Y_91:t, X_92:t] : ((mul_8e @ (mul_8e @ X_92 @ X_92) @ Y_91) = (mul_8e @ Y_91 @ X_92)))).
thf(ax_1, axiom, ~((![Y_91:t, X_92:t] : ((mul_8e @ X_92 @ Y_91) = (mul_8e @ Y_91 @ X_92))))).


In [9]:
s = solvers.EProverTHFSolver()
s.add(smt.ForAll([x,y,z], (x * x) * y == y * x))
s.add(smt.Not(smt.ForAll([x,y], x * y == y * x)))
s.set("timeout", 1000)
print(s.check())
print(s.proof().decode())

unsat
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 2 seconds (16 total)
# Starting new_ho_10 with 10s (5) cores
# Starting ho_unfolding_6 with 2s (1) cores
# Starting sh4l with 2s (1) cores
# Starting ehoh_best_nonlift_rwall with 2s (1) cores
# sh4l with pid 2515392 completed with status 0
# Result found by sh4l
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 2 seconds (16 total)
# Starting new_ho_10 with 10s (5) cores
# Starting ho_unfolding_6 with 2s (1) cores
# Starting sh4l with 2s (1) cores
# No SInE strategy applied
# Search class: HUUPF-FFSF22-SFFFFFNN
# Scheduled 1 strats onto 1 cores with 2 seconds (2 total)
# Starting new_ho_10 with 2s (1) cores
# new_ho_10 with pid 2515394 completed with status 0
# Result found by new_ho_10
# Preprocessing class: HSSSSMSSSSSNFFN.
# Scheduled 4 strats onto 8 cores with 2 seconds (16 total)
# Starting new_ho_10 with 10s (5) cores
# Starting ho_unfolding_6 with 2s (1) cores
# Star

In [23]:
! cat /tmp/eprover.p

% TPTP file generated by Knuckledragger
thf(t_type, type, t : $tType ).
% Declarations
thf(mul_decl, type, mul_8e : t > t > t).
% Axioms and assertions
thf(ax_0, axiom, (![Z_90:t, Y_91:t, X_92:t] : ((mul_8e @ (mul_8e @ X_92 @ X_92) @ Y_91) = (mul_8e @ Y_91 @ X_92)))).
thf(ax_1, axiom, ~((![Y_91:t, X_92:t] : ((mul_8e @ X_92 @ Y_91) = (mul_8e @ Y_91 @ X_92))))).


# What's Next
It seems there is already code to enumerate possible axioms made of a multiplication operation. Should do that.